# How to create a session object, and work with it

In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

# TODO
* Nikon SW time for high frequency recordings seems to be slow (i.e. delayed). This time is used by labview!

## Imports

In [ ]:
# import modules from 2p-py
import labrotation.file_handling as file_handling
import labrotation.two_photon_session as two_photon_session
import labrotation.belt_processing as belt_processing
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Helpful functions and parameters

In [ ]:
def get_datetime_for_fname():
    now = datetime.now()
    return f"{now.year:04d}{now.month:02d}{now.day:02d}-{now.hour:02d}{now.minute:02d}{now.second:02d}"

In [ ]:
save_figs = False  # set to True to save the figures created
save_as_eps = True
if save_as_eps:
    file_format = ".eps"
else:
    file_format = ".jpg"
if save_figs:
    print(f"Going to save figures as {file_format} files.")

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

## Set matlab-2p path
matlab is unfortunately still needed, as the belt processing scripts were written in Matlab and have not yet been implemented in Python. See the Test Matlab Integration jupyter notebook on how to set up Matlab in Python.

In [ ]:
matlab_2p_path = env_dict["MATLAB_2P_FOLDER"]#file_handling.open_dir("Choose matlab-2p folder!")

## Set file path variables

In [ ]:
nd2_path = file_handling.open_file("Choose Nikon imaging file (.nd2)!")
nd2_meta_path = file_handling.open_file("Choose Nikon metadata file (.txt)!")
labview_path = file_handling.open_file("Choose LabView file (xy.txt, NOT xytime.txt)!")
labview_timestamps_path = labview_path[:-4] + "time.txt"
lfp_path = file_handling.open_file("Choose LFP file (.abf)!")

In [ ]:
nd2_fname = os.path.splitext(os.path.split(nd2_path)[-1])[0]

In [ ]:
# open a file as time.txt that is not in the same folder, and/or does not have the usual xy.txt -> xytime.txt format.
different_timetxt = False  
if different_timetxt:
    labview_timestamps_path = file_handling.open_file("Choose Labview time stamps (time.txt) file")

In [ ]:
if lfp_path == ".":
    lfp_path = None
if labview_path == ".":
    labview_path = None
    labview_timestamps_path = None

## Create session object
Functions such as matching the belt with the nikon recording will be automatically run.

In [ ]:
session = two_photon_session.TwoPhotonSession.init_and_process(nd2_path, nd2_meta_path, labview_path, labview_timestamps_path, lfp_path, matlab_2p_path)

In [ ]:
session.time_offs_lfp_nik

In [ ]:
#session.shift_lfp(1.5, match_type="zero")

In [ ]:
nik_means = session.mean_fluo

In [ ]:
nik_t = session.nikon_daq_time.values

In [ ]:
nik_sw_t = session.nikon_meta["SW Time [s]"].values

In [ ]:
nik_sw_t

In [ ]:
if nik_sw_t[0].dtype == np.float64:
    pass
else:
    nik_sw_t = np.array(list(map(lambda x: float(x.replace(",", ".")), nik_sw_t)))

In [ ]:
ar = nik_sw_t[1:] - nik_sw_t[:-1]

In [ ]:
len_rec = nik_sw_t[-1] - nik_sw_t[0]  # the real length of the recording in seconds

In [ ]:
# nik_t: 0 to b, N frames. nik_sw_t: c to d, N frames.
# approximate time as software time (problem: difference between elements is not constant)
nik_t_approx_s = nik_sw_t - nik_sw_t[0]

In [ ]:
session.belt_scn_dict.keys()

## session is ready to work with!
Check class documentation (or source code) for functionalities

In [ ]:
movement_t, movement_y = session.lfp_movement(as_numpy=True)  # split components
movement = session.lfp_movement()  # keep components together

In [ ]:

fig = plt.figure(figsize=(18,10))
plt.plot(session.belt_scn_dict["tsscn"]/1000., session.belt_scn_dict["speed"])
plt.plot(movement_t, movement_y)  # or: plt.plot(*movement)
plt.xlim((15, 25))
plt.show()

In [ ]:
lfp_t, lfp_y = session.lfp_lfp(as_numpy=True)

In [ ]:
fig = plt.figure(figsize=(18,10))
plt.plot(lfp_t,lfp_y)  # or: plt.plot(*movement)
plt.show()

In [ ]:
fig, axs = plt.subplots(3,1,figsize=(18,18), sharex=True)
axs[0].plot(nik_t_approx_s, nik_means)
axs[1].plot(session.belt_scn_dict["tsscn"]/1000., session.belt_scn_dict["speed"], color="grey")
axs[2].plot(movement_t, movement_y, color="green")
axs[2].set_ylim((2.0, 3.5))
#axs[0].set_ylim((42.0, 50.))
#axs[2].set_xlim((18.0, 30))
axs[2].set_xlim((15, 25))

plt.show()

In [ ]:
nik_ymax = 58
nik_ymin = 38

fig, axs = plt.subplots(2,1,figsize=(26,18), sharex=True)
axs[0].plot(nik_t_approx_s, nik_means)
axs[0].set_ylim((nik_ymin,nik_ymax))
axs[1].plot(session.belt_scn_dict["tsscn"]/1000., session.belt_scn_dict["speed"], color="grey")
#axs[3].set_xlim((18,70))
plt.show()

In [ ]:
nik_ymax = 58
nik_ymin = 38

fig, axs = plt.subplots(2,1,figsize=(26,18), sharex=True)
axs[0].plot(nik_t_approx_s, nik_means)
axs[0].set_ylim((nik_ymin,nik_ymax))
axs[1].plot(lfp_t, lfp_y, color="green")
axs[1].set_xlim((18,70))
plt.show()

In [ ]:
nik_ymax = 58
nik_ymin = 38


fig, axs = plt.subplots(4,1,figsize=(26,18), sharex=True)
axs[0].plot(nik_t_approx_s, nik_means)
axs[0].set_ylim((nik_ymin,nik_ymax))
axs[1].plot(lfp_t, lfp_y, color="green")
axs[2].plot(movement_t, movement_y, color="grey")
axs[2].set_ylim((2.0,3.5))
axs[3].plot(session.belt_scn_dict["tsscn"]/1000., session.belt_scn_dict["speed"], color="grey")
#axs[3].set_xlim((18,70))
axs[3].set_xlim((175,200))

if save_figs and True:
    out_fpath = f"D:\\Downloads\\{nd2_fname}_whole_{get_datetime_for_fname()}{file_format}"
    plt.savefig(out_fpath)
    print(f"Saved to {out_fpath}")
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,12))
plt.plot(session.belt_dict["speed"])
plt.xlim((14000, 15000))
plt.show()

In [ ]:
output_folder = file_handling.open_dir("Choose output folder!")

In [ ]:
save_matched_lfp = False
if save_matched_lfp:
    with open(os.path.join(output_folder, "lfp_matched.txt"), "w") as f:
        for i_row in range(len(lfp_t)):
            f.write(f"{lfp_t[i_row]}\t{lfp_y[i_row]}\n")

In [ ]:
save_matched_movement = False
if save_matched_movement:
    with open(os.path.join(output_folder, "movement_matched.txt"), "w") as f:
        for i_row in range(len(movement_t)):
            f.write(f"{movement_t[i_row]}\t{movement_y[i_row]}\n")

In [ ]:
save_nik = False
if save_nik:
    with open(os.path.join(output_folder, "nik_matched.txt"), "w") as f:
        for i_row in range(len(nik_means)):
            f.write(f"{nik_t_approx_s[i_row]}\t{nik_means[i_row]}\n")

# Save data to matlab .mat file

In [ ]:
import scipy

In [ ]:
matlab_dict = {"nik_t": nik_t_approx_s, "nik_mean": nik_means, "lfp_t": lfp_t, "lfp_y": lfp_y, "lfp_mov_t": movement_t, "lfp_mov_y": movement_y, "lv_mov_t": session.belt_scn_dict["tsscn"]/1000., "lv_mov_y": session.belt_scn_dict["speed"]}

In [ ]:
matlab_dict

In [ ]:
scipy.io.savemat(f"D:\\Downloads\\{nd2_fname}_traces.mat", matlab_dict)